In [ ]:
# Installation cell for Google Colab (due to Stenetorp/Bradbury; bumped version!)
# see https://colab.research.google.com/drive/1gMhnrMNP-het4rexXLXHzaOytfDpUxgR
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.2/julia-1.2.0-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'

In [1]:
import Pkg; Pkg.add("CUDAnative"); Pkg.add("CuArrays");
using Random
using CuArrays
using CUDAnative
using Statistics

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %41.0 %>       ]  82.3 % Resolving package versions...
  Updating `~/Google Drive/Tufts/MGGG/OptimalVotes.jl/Project.toml`
 [no changes]
  Updating `~/Google Drive/Tufts/MGGG/OptimalVotes.jl/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Google Drive/Tufts/MGGG/OptimalVotes.jl/Project.toml`
 [no changes]
  Updating `~/Google Drive/Tufts/MGGG/OptimalVotes.jl/Manifest.toml`
 [no changes]


┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/pjrule/.julia/packages/CUDAdrv/aBgcd/src/CUDAdrv.jl:69


In [2]:
# Hack to enable CUDA development on non-CUDA-enabled machines:
# https://juliagpu.gitlab.io/CUDA.jl/installation/conditional/#Scenario-2:-GPU-is-optional-1
if CuArrays.functional()
    CuArrays.allowscalar(false)
    gpu(x::AbstractArray) = CuArray(x)
else
    gpu(x::AbstractArray) = x
end

gpu (generic function with 1 method)

In [52]:
struct GridEnsemble
    width::Int
    height::Int
    size::Int
    n_districts::Int
    district_pop::Int
    weights::AbstractArray{Int}
    plans::AbstractArray{Int, 2}
    n_plans::Int
end

"""Load enumerated plans from a @zschutzman-style CSV."""
function GridEnsemble(plans_file::AbstractString, width::Int, height::Int, n_districts::Int)
    n_plans = countlines(plans_file)
    plans = zeros(Int, n_plans, width * height)
    weights = ones(Int, n_plans)
    
    open(plans_file) do file
        for (plan_idx, plan) in enumerate(eachline(file))
            assignments = [parse(Int, a) for a in split(plan, ",")]
            @assert length(assignments) == width * height
            @simd for i in 1:length(assignments)
                plans[plan_idx, i] = assignments[i]
            end
        end
    end
    
    district_pop = Int((width * height) / n_districts)
    plan_size = width * height
    return GridEnsemble(width, height, plan_size, n_districts, district_pop,
                        gpu(weights), gpu(plans), size(plans, 1))
end

GridEnsemble

In [53]:
"""Generate all neighbors to the current vote distribution `baseline` ∈ Δ.

    Neighbors are formed by moving one voter left, right, up, or down one unit.
"""
function Δ_neighbors(baseline::Array{Int}, width::Int, height::Int)::Set{Array{Int}}
    neighbors = Set{Array{Int}}()
    baseline_grid = reshape(baseline, width, height)
    for row in 1:height
        for col in 1:width
            if baseline_grid[row, col] == 1
                perturbed = copy(baseline_grid)
                perturbed[row, col] = 0
                left = copy(perturbed)
                left[row, max(1, col - 1)] = 1
                right = copy(perturbed)
                left[row, min(col, width)] = 1
                up = copy(perturbed)
                up[max(1, row - 1), col] = 1
                down = copy(perturbed)
                down[min(row, height), col] = 1
                push!(neighbors, flatten(left), flatten(right),
                      flatten(up), flatten(down))
            end
        end
    end
    pop!(neighbors, baseline)
    return neighbors
end

Δ_neighbors

In [71]:
function kernel_vote_share(plans::CuArray{Int, 2},
                           δ::CuArray{Int},
                           results_buf::CuArray{Int},
                           params::CuArray{Int})
    # params[0] – plan size
    # params[1] – district pop
    # params[2] – number of districts
    plan_idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    vote_counts = CuArrays.zeros(param[2])  # TODO: will this work?
    
    win_count = 0
    for i in params[0]
        vote_counts[plans[plan_idx, i]] += δ[i]
    end
    for i in 1:params[2]
        if vote_counts[i] / params[1] > 0.5
            win_count += 1
        elseif vote_counts[i] / params[1] == 0.5
            win_count += 0.5
        end
    end
    results_buf[plan_idx] = win_count
end

function vote_share(plan::Array{Int}, δ::Array{Int}, district_pop::Int, n_districts::Int)
    vote_counts = zeros(Int, n_districts)
    for i in 1:length(plan)
        @inbounds vote_counts[plan[i]] += δ[i]
    end
    win_count = 0
    for count in vote_counts
        if count / district_pop > 0.5
            win_count += 1
        elseif count / district_pop == 0.5
            win_count += 0.5
        end
    end
    return win_count
end

vote_share (generic function with 1 method)

In [72]:
function expected_vote_share(ensemble::GridEnsemble, δ::Array)::Float64
    n_plans = length(ensemble.plans)
    share = zeros(Float64, n_plans)
    if CuArrays.functional()
        results_gpu = CuArrays.zeros(ensemble.n_plans)
        params = gpu([ensemble.size, ensemble.district_pop,
                      ensemble.n_districts])
        @cuda threads=2048 kernel_vote_share(plans, gpu(δ), results_gpu, params)
        results = Array(results_gpu)
    else
        results = [vote_share(ensemble.plans[i, :], δ, ensemble.district_pop,
                              ensemble.n_districts) for i in 1:ensemble.n_plans]
    end
    return mean(results)
end

expected_vote_share (generic function with 1 method)

In [66]:
"""Maximize expected vote share while maintaining partisan split."""
function anneal(ensemble::GridEnsemble, baseline::Array, objective::Function, temp::Float64, iterations::Int)
    best_δ = copy(baseline)
    curr_δ = copy(baseline)
end

anneal

In [67]:
ensemble = GridEnsemble("enum.csv", 6, 6, 6);
δ = zeros(Int, 36);
δ[1:12] .= 1;

In [78]:
@time expected_vote_share(ensemble, δ)

  0.529087 seconds (3.48 M allocations: 383.603 MiB, 32.39% gc time)


1.9137733097520866

In [33]:
ensemble.n_plans

16243416